<div class="alert alert-warning not-in-docs">
⚠️ Links are not working in the notebook. Please visit <a href="https://qhyper.readthedocs.io/en/latest/user_guide/demo/defining_problems.html">documentation</a> for better experience.
</div>

# Defining custom problems, optimizers and solvers

## A user-defined class

A big advantage of using QHyper is the ability to run experiments from a configuration file.
However, this only allows to use predefined problems, optimizers and solvers. In this notebook, we present a concise example illustrating how to define a custom problem, although the same principles apply to custom optimizers and solvers. We have chosen to highlight problem definition, as it is likely one of the most practical and valuable use cases for QHyper.

.. note::
    Any custom [Problem](../../generated/QHyper.problems.rst), [Optimizer](../../generated/QHyper.optimizers.rst) or [Solver](../../generated/QHyper.solvers.rst) class should be implemented in the directory named `custom` or `QHyper/custom`. It is required that these classes inherit from their base classes and implement required methods. The new class will be available in configuration files by its attribute name if provided or by its class name.

## Creating a custom problem

Assume we want to  minimize $\underbrace{-2x_0 - 5x_1 - x_0x_1}_{cost function}$ subject to $\underbrace{x_0 + x_1 = 1}_{constraint\ eq}$ and $\underbrace{5x_0 + 2x_1 \leq 5}_{constraint\ le}$

In QHyper, every problem needs to be a subclass of the [Problem class](../../generated/QHyper.problems.rst).

In general, the cost function and every constraint should be expressed as dict-based [Polynomials](../../generated/QHyper.polynomial.rst), but usually it is easier to initially express them in a more user-friendly format (such as SymPy syntax), and then convert it them into Polynomials. A Polynomial is comprised of a dictionary where the keys are tuples containing variables, and the values represent their coefficients.

To define the constraints, the [Constraint](../../generated/QHyper.constraint.rst) class is used. Each constraint involves Polynomials on the left-hand side (lhs) and right-hand side (rhs), a comparison operator, and optional data such as a method for handling inequalities, a label, and a group identifier.

.. note::
    QHyper always assumes that the objective is to **minimize** the cost function.

### Using Dict syntax

In [1]:
import numpy as np

from QHyper.problems.base import Problem
from QHyper.constraint import Constraint, Operator, UNBALANCED_PENALIZATION
from QHyper.polynomial import Polynomial

class CustomProblem(Problem):
    def __init__(self) -> None:
        self.objective_function = self._create_objective_function()
        self.constraints = self._create_constraints()

    def _create_objective_function(self) -> Polynomial:
        # Express the cost function as a dict. The keys are tuples containing variables, and the values represent the coefficients.
        objective_function = {('x0',): -2.0, ('x1',): -5.0, ('x0', 'x1'): -1.0}
        
        # Create a Polynomial based on the objective function.
        return Polynomial(objective_function)

    def _create_constraints(self) -> list[Constraint]:
        # To add a new constraint, define the left-hand-side, and right-hand-side of the constraint.
        # Also, specify the comparison operator and in the case of inequality opertor --- the method for handling the inequality.
        
        constraints = [
            Constraint(lhs={('x0',): 1.0, ('x1',): 1.0}, rhs={(): 1},
                       operator=Operator.EQ),
            Constraint(lhs={('x0',): 5.0, ('x1',): 2.0}, rhs={(): 5}, 
                       operator=Operator.LE,
                       method_for_inequalities=UNBALANCED_PENALIZATION)
        ]
        return constraints

    def get_score(self, result: np.record, penalty: float = 0) -> float:
        # This function is used by solvers to evaluate the quality of the result (business value).
        
        # If the constraints are satisfied return the value of the objective function.
        if result['x0'] + result['x1'] == 1 and 5 * result['x0'] + 2 * result['x1'] <= 5:
            return -2 * result['x0'] - 5 * result['x1'] - result['x0'] * result['x1']
        
        # Otherwise return some arbitrary penalty
        return penalty

### Using SymPy syntax

In [2]:
import sympy
import numpy as np
from QHyper.problems.base import Problem
from QHyper.constraint import Constraint, Operator, UNBALANCED_PENALIZATION
from QHyper.polynomial import Polynomial
from QHyper.parser import from_sympy


class CustomProblem(Problem):
    def __init__(self) -> None:
        # Define the necessary SymPy variables.
        num_variables = 2
        self.x = sympy.symbols(f'x0:{num_variables}')
        self.objective_function = self._create_objective_function()
        self.constraints = self._create_constraints()

    def _create_objective_function(self) -> Polynomial:
        # Define the cost function.
        objective_function = -2 * self.x[0] - 5 * self.x[1] - self.x[0] * self.x[1]
        
        # Return the cost function parsed into a Polynomial
        return from_sympy(objective_function)

    def _create_constraints(self) -> list[Constraint]:
        # To add a new constraint, define the left-hand-side, and right-hand-side of the constraint.
        # Also, specify the comparison operator and in the case of inequality opertor --- the method for handling the inequality.
        
        return [ 
            Constraint( 
                lhs=from_sympy(self.x[0] + self.x[1]),
                rhs=1,
                operator=Operator.EQ
            ),
            Constraint(
                lhs=from_sympy(5 * self.x[0] + 2 * self.x[1]),
                rhs=5,
                operator=Operator.LE,
                method_for_inequalities=UNBALANCED_PENALIZATION,
            )
        ]

    def get_score(self, result: np.record, penalty: float = 0) -> float:
        # This function is used by solvers to evaluate the quality of the result (business value).
        
        # If the constraints are satisfied return the value of the objective function.
        if result['x0'] + result['x1'] == 1 and 5 * result['x0'] + 2 * result['x1'] <= 5:
            return -2 * result['x0'] - 5 * result['x1'] - result['x0'] * result['x1']
        
        # Otherwise return some arbitrary penalty
        return penalty

.. note::
    For bigger problem instances, SymPy syntax is significantly slower than the Dict syntax.

To explore more solvers for tackling this problem, check out this [tutorial](../solver_configuration.rst).